In [ ]:
import pandas as pd
import os

In [ ]:
#set directory
os.chdir('/Users/kl/My Drive/Research/GPT Patents')

In [ ]:
hof = pd.read_csv('National Inventors Hall of Fame/hof.csv')

In [ ]:
eval(hof['patent'].loc[0])

In [ ]:
#open xlsx file
kelly_patents = pd.read_excel('Kelly et al. (2021)/Kelly et al. (2021) Important Patents.xlsx')

In [ ]:
kelly_patents['kellyetal'] = 1
kelly_patents['halloffame'] = 0
kelly_patents['Patent'] = kelly_patents['Patent'].astype(str)

In [ ]:
for i in range(len(hof)):
    if hof['patent'].loc[i] == 'missing':
        continue
    patlist = eval(hof['patent'].loc[i])
    for patent in patlist:
        patnum = patent.replace(',','')
        if patnum in kelly_patents['Patent'].tolist():
            kelly_patents.loc[kelly_patents['Patent'] == patnum, 'halloffame'] = 1
        else:
            kelly_patents = pd.concat([kelly_patents, pd.DataFrame({'Patent': [patnum], 'Year': [None], 'Inventor': [hof['name'].loc[i]],'Invention': [hof['invention'].loc[i]],'kellyetal': [0], 'halloffame': [1]})], axis=0)
        


In [ ]:
kelly_patents.reset_index(drop=True, inplace=True)

In [ ]:
kelly_patents

In [ ]:
kelly_patents[kelly_patents['halloffame']*kelly_patents['kellyetal'] == 1]

In [ ]:
kelly_patents.to_csv('significant_patents.csv', index=False)

In [ ]:
kelly_patents['Patent'][0]

In [ ]:
pd.read_csv('/Users/kl/Downloads/g_us_patent_citation.tsv', delimiter='\t', nrows=10)['citation_patent_id']

In [ ]:
read = pd.read_csv('/Users/kl/Downloads/g_us_patent_citation.tsv', delimiter='\t', usecols=['patent_id','citation_patent_id'], chunksize=1000000)
patlist = list(kelly_patents['Patent'].unique())

bwd = pd.concat(chunk[chunk['patent_id'].astype(str).isin(patlist)] for chunk in read)
fwd = pd.concat(chunk[chunk['citation_patent_id'].astype(str).isin(patlist)] for chunk in read)


In [ ]:
bwd = pd.DataFrame()
fwd = pd.DataFrame()
read = pd.read_csv('/Users/kl/Downloads/g_us_patent_citation.tsv', delimiter='\t', usecols=['patent_id','citation_patent_id'], chunksize=1000000)
for chunk in read:
    chunk['citation_patent_id'] = chunk['citation_patent_id'].astype(str)
    chunk['patent_id'] = chunk['patent_id'].astype(str)
    bwd = pd.concat([bwd, chunk[chunk['patent_id'].isin(patlist)]])
    fwd = pd.concat([fwd, chunk[chunk['citation_patent_id'].isin(patlist)]])

In [ ]:
fwd['citation_patent_id'].nunique()

In [ ]:
bwd['patent_id'].nunique()

In [ ]:
fwd_sub = fwd[fwd['citation_patent_id'].isin(bwd['patent_id'])]

In [ ]:
fwd_sub['patent_id'].nunique()

In [ ]:
fwd_sub['patent_id'].iloc[0]

In [ ]:
bwd_of_fwd = pd.DataFrame()
read = pd.read_csv('/Users/kl/Downloads/g_us_patent_citation.tsv', delimiter='\t', usecols=['patent_id','citation_patent_id'], chunksize=1000000)
for chunk in read:
    chunk['citation_patent_id'] = chunk['citation_patent_id'].astype(str)
    chunk['patent_id'] = chunk['patent_id'].astype(str)
    bwd_of_fwd = pd.concat([bwd_of_fwd, chunk[chunk['patent_id'].isin(fwd_sub['patent_id'])]])

In [ ]:
bwd_of_fwd.patent_id.nunique()

In [ ]:
fwd_sub['fwd_cit_bwd'] = 0
for i in range(len(fwd_sub)):
    if i==0 or fwd_sub['citation_patent_id'].iloc[i] != fwd_sub['citation_patent_id'].iloc[i-1]:
        focal = fwd_sub['citation_patent_id'].iloc[i]
        bwd_list = bwd[bwd['patent_id'] == focal]['citation_patent_id'].tolist()
    citing = fwd_sub['patent_id'].iloc[i]
    fwd_bwd_list = bwd_of_fwd[bwd_of_fwd['patent_id'] == citing]['citation_patent_id'].tolist()
    if len(set(bwd_list).intersection(fwd_bwd_list)) > 0:
        fwd_sub['fwd_cit_bwd'].iloc[i] = 1
    if i % 1000 == 0:
        print(i)


In [ ]:
fwd_sub.to_csv('fwd_sub.csv', index=False)
fwd.to_csv('fwd.csv', index=False)
bwd.to_csv('bwd.csv', index=False)
bwd_of_fwd.to_csv('bwd_of_fwd.csv', index=False)

In [ ]:
fwd_sub.iat[0,'citation_patent_id']

In [ ]:
from openai import OpenAI

In [ ]:
import pandas as pd

In [ ]:
patents = pd.read_csv('/Users/kl/Downloads/g_patent.tsv', delimiter='\t')

In [ ]:
patents

In [ ]:
patents['patent_type'].unique()

In [ ]:
patents[(patents['patent_type'] == 'design') & (patents['patent_id'].str[:1]=='D')]